## First we install Python packages and initialize our globals

In [1]:
%pip install -r ../requirements.txt
%load_ext autoreload
%autoreload 2
# add project dir to python path
try:
    notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
    import sys, os
    sys.path.append("/Workspace" + os.path.dirname(os.path.dirname(notebook_path)))
except:
    pass

Note: you may need to restart the kernel to use updated packages.


In [3]:
# load table into spark dataframe
cows_bff = spark.read.table("p2dscidmsmeus.glestryc.cows_bff")

## Lets compute meal overlap

In [4]:
from cow_bff.heatmap import compute_heatmap

df = compute_heatmap(cows_bff)
display(df.limit(10))

,cow1,cow2,closeness
0,Brandy,Buttercup,687.000000
1,Brandy,Butterscotch,627.461538
2,Brandy,Cocoa,418.692308
3,Brandy,Daisy,406.846154
4,Brandy,Dottie,723.538462
5,Brandy,Hershey,507.384615
6,Brandy,Magic,319.153846
7,Brandy,Marshmallow,454.846154
8,Brandy,Mocha,418.692308
9,Brandy,Muffin,338.692308


# Display Heatmap

In [5]:
pdf = df.toPandas()
pdf = pdf.pivot(index='cow1', columns='cow2', values='closeness').fillna(0)

import plotly.express as px
fig = px.imshow(pdf, x=pdf.columns, y=pdf.index, labels=dict(x="Cow 2", y="Cow 1", color="closeness"), title="Cow BFFs", color_continuous_scale='redor')
px.imshow(pdf, x=pdf.columns, y=pdf.index, labels=dict(x="Cow 2", y="Cow 1", color="closeness"),)

fig.update_layout(width=800,height=500)
fig.show()

In [6]:
display(df)

,cow1,cow2,closeness
0,Brandy,Buttercup,687.000000
1,Brandy,Butterscotch,627.461538
2,Brandy,Cocoa,418.692308
3,Brandy,Daisy,406.846154
4,Brandy,Dottie,723.538462
5,Brandy,Hershey,507.384615
6,Brandy,Magic,319.153846
7,Brandy,Marshmallow,454.846154
8,Brandy,Mocha,418.692308
9,Brandy,Muffin,338.692308
